# Quickstart

Heat a small workshop with a gas boiler - the minimal working example.

This notebook introduces the **core concepts** of flixopt:

- **FlowSystem**: The container for your energy system model
- **Bus**: Balance nodes where energy flows meet
- **Effect**: Quantities to track and optimize (costs, emissions)
- **Components**: Equipment like boilers, sources, and sinks
- **Flow**: Connections between components and buses

## Setup

In [1]:
import pandas as pd
import plotly.express as px
import xarray as xr

import flixopt as fx

fx.CONFIG.notebook()

flixopt.config.CONFIG

## Define the Time Horizon

Every optimization needs a time horizon. Here we model a simple 4-hour period:

In [2]:
timesteps = pd.date_range('2024-01-15 08:00', periods=4, freq='h')
print(f'Optimizing from {timesteps[0]} to {timesteps[-1]}')

Optimizing from 2024-01-15 08:00:00 to 2024-01-15 11:00:00


## Define the Heat Demand

The workshop has varying heat demand throughout the morning:

In [3]:
# Heat demand in kW for each hour - using xarray
heat_demand = xr.DataArray(
    [30, 50, 45, 25],
    dims=['time'],
    coords={'time': timesteps},
    name='Heat Demand [kW]',
)

# Visualize the demand with plotly
fig = px.bar(x=heat_demand.time.values, y=heat_demand.values, labels={'x': 'Time', 'y': 'Heat Demand [kW]'})
fig

## Build the Energy System Model

Now we create the FlowSystem and add all components:

```
  Gas Supply ──► [Gas Bus] ──► Boiler ──► [Heat Bus] ──► Workshop
       €                         η=90%                    Demand
```

In [4]:
# Create the FlowSystem container
flow_system = fx.FlowSystem(timesteps)

flow_system.add_elements(
    # === Buses: Balance nodes for energy carriers ===
    fx.Bus('Gas'),  # Natural gas network connection
    fx.Bus('Heat'),  # Heat distribution within workshop
    # === Effect: What we want to minimize ===
    fx.Effect('costs', '€', 'Total Costs', is_standard=True, is_objective=True),
    # === Gas Supply: Unlimited gas at 0.08 €/kWh ===
    fx.Source(
        'GasGrid',
        outputs=[fx.Flow('Gas', bus='Gas', size=1000, effects_per_flow_hour=0.08)],
    ),
    # === Boiler: Converts gas to heat at 90% efficiency ===
    fx.linear_converters.Boiler(
        'Boiler',
        thermal_efficiency=0.9,
        thermal_flow=fx.Flow('Heat', bus='Heat', size=100),  # 100 kW capacity
        fuel_flow=fx.Flow('Gas', bus='Gas'),
    ),
    # === Workshop: Heat demand that must be met ===
    fx.Sink(
        'Workshop',
        inputs=[fx.Flow('Heat', bus='Heat', size=1, fixed_relative_profile=heat_demand.values)],
    ),
)

## Run the Optimization

Now we solve the model using the HiGHS solver (open-source, included with flixopt):

In [5]:
flow_system.optimize(fx.solvers.HighsSolver());

## Analyze Results

### Heat Balance

The `statistics.plot.balance()` method shows how each bus is balanced:

In [6]:
flow_system.statistics.plot.balance('Heat')

PlotResult(data=<xarray.Dataset> Size: 120B
Dimensions:         (time: 5)
Coordinates:
  * time            (time) datetime64[ns] 40B 2024-01-15T08:00:00 ... 2024-01...
Data variables:
    Boiler(Heat)    (time) float64 40B -30.0 -50.0 -45.0 -25.0 nan
    Workshop(Heat)  (time) float64 40B 30.0 50.0 45.0 25.0 nan, figure=Figure({
    'data': [{'fillcolor': '#EF553B',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f}<extra></extra>',
              'legendgroup': 'Boiler(Heat)',
              'line': {'color': '#EF553B', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Boiler(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'negative',
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAPsAAAAAAAABJwAAAAAAAgEbAAAAAAAAAOcAAAAAAAAD4/w==', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'fillcolor': '#00CC96',
              'fillpattern': {'shape': ''},
              'hovertemplate': '<b>%{fullData.name}</b>: %{y:.1f}<extra></extra>',
              'legendgroup': 'Workshop(Heat)',
              'line': {'color': '#00CC96', 'shape': 'hv', 'width': 0},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Workshop(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'stackgroup': 'positive',
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAPkAAAAAAAABJQAAAAAAAgEZAAAAAAAAAOUAAAAAAAAD4fw==', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'hovermode': 'x unified',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Heat'},
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'showspikes': True,
                         'spikecolor': 'gray',
                         'spikethickness': 1,
                         'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
}))

### Total Costs

Access the optimized objective value:

In [7]:
total_costs = flow_system.solution['costs'].item()
total_heat = float(heat_demand.sum())
gas_consumed = total_heat / 0.9  # Account for boiler efficiency

print(f'Total heat demand: {total_heat:.1f} kWh')
print(f'Gas consumed: {gas_consumed:.1f} kWh')
print(f'Total costs: {total_costs:.2f} €')
print(f'Average cost: {total_costs / total_heat:.3f} €/kWh_heat')

Total heat demand: 150.0 kWh
Gas consumed: 166.7 kWh
Total costs: 13.33 €
Average cost: 0.089 €/kWh_heat


### Flow Rates Over Time

Visualize all flow rates using the built-in plotting accessor:

In [8]:
# Plot all flow rates
flow_system.statistics.plot.flows()

PlotResult(data=<xarray.Dataset> Size: 200B
Dimensions:         (time: 5)
Coordinates:
  * time            (time) datetime64[ns] 40B 2024-01-15T08:00:00 ... 2024-01...
Data variables:
    Boiler(Gas)     (time) float64 40B 33.33 55.56 50.0 27.78 nan
    Boiler(Heat)    (time) float64 40B 30.0 50.0 45.0 25.0 nan
    GasGrid(Gas)    (time) float64 40B 33.33 55.56 50.0 27.78 nan
    Workshop(Heat)  (time) float64 40B 30.0 50.0 45.0 25.0 nan, figure=Figure({
    'data': [{'hovertemplate': 'variable=Boiler(Gas)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Boiler(Gas)',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Boiler(Gas)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'q6qqqqqqQEByHMdxHMdLQAAAAAAAAElAchzHcRzHO0AAAAAAAAD4fw==', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Boiler(Heat)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Boiler(Heat)',
              'line': {'color': '#EF553B', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Boiler(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAPkAAAAAAAABJQAAAAAAAgEZAAAAAAAAAOUAAAAAAAAD4fw==', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=GasGrid(Gas)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'GasGrid(Gas)',
              'line': {'color': '#636EFA', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'GasGrid(Gas)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'q6qqqqqqQEByHMdxHMdLQAAAAAAAAElAchzHcRzHO0AAAAAAAAD4fw==', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=Workshop(Heat)<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'Workshop(Heat)',
              'line': {'color': '#00CC96', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Workshop(Heat)',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array(['2024-01-15T08:00:00.000000000', '2024-01-15T09:00:00.000000000',
                          '2024-01-15T10:00:00.000000000', '2024-01-15T11:00:00.000000000',
                          '2024-01-15T12:00:00.000000000'], dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAPkAAAAAAAABJQAAAAAAAgEZAAAAAAAAAOUAAAAAAAAD4fw==', 'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title

### Energy Flow Sankey

A Sankey diagram visualizes the total energy flows through the system:

In [9]:
flow_system.statistics.plot.sankey.flows()

PlotResult(data=<xarray.Dataset> Size: 560B
Dimensions:  (link: 4)
Coordinates:
  * link     (link) int64 32B 0 1 2 3
    source   (link) <U7 112B 'Gas' 'Boiler' 'GasGrid' 'Heat'
    target   (link) <U8 128B 'Boiler' 'Heat' 'Gas' 'Workshop'
    label    (link) <U14 224B 'Boiler(Gas)' 'Boiler(Heat)' ... 'Workshop(Heat)'
    carrier  (link) object 32B None None None None
Data variables:
    value    (link) float64 32B 166.7 150.0 166.7 150.0, figure=Figure({
    'data': [{'link': {'color': [rgba(200, 200, 200, 0.4), rgba(200, 200, 200,
                                 0.4), rgba(200, 200, 200, 0.4), rgba(200, 200,
                                 200, 0.4)],
                       'label': [Boiler(Gas), Boiler(Heat), GasGrid(Gas),
                                 Workshop(Heat)],
                       'source': [3, 0, 4, 2],
                       'target': [0, 2, 3, 1],
                       'value': [166.66666666666666, 150.0, 166.66666666666666,
                                 150.0]},
              'node': {'color': [#EF553B, #00CC96, #636EFA, #EF553B, #636EFA],
                       'label': [Boiler, Workshop, Heat, Gas, GasGrid],
                       'line': {'color': 'black', 'width': 0.5},
                       'pad': 15,
                       'thickness': 20},
              'type': 'sankey'}],
    'layout': {'template': '...', 'title': {'text': 'Energy Flow'}}
}))

## Summary

In this quickstart, you learned the **basic workflow**:

1. **Create** a `FlowSystem` with timesteps
2. **Add** buses, effects, and components
3. **Optimize** with `flow_system.optimize(solver)`
4. **Analyze** results via `flow_system.statistics`

### Next Steps

- **[02-heat-system](02-heat-system.ipynb)**: Add thermal storage to shift loads
- **[03-investment-optimization](03-investment-optimization.ipynb)**: Optimize equipment sizing